# 单动作多智能体

MetaGPT的核心优势也在于轻松灵活地开发一个智能体团队。

我们需要三个步骤来建立团队并使其运作：

定义每个角色能够执行的预期动作

基于标准作业程序（SOP）确保每个角色遵守它。通过使每个角色观察上游的相应输出结果，并为下游发布自己的输出结果，可以实现这一点。

初始化所有角色，创建一个带有环境的智能体团队，并使它们之间能够进行交互。

内容来自于：
[https://docs.deepwisdom.ai/v0.8/zh/guide/tutorials/multi\_agent\_101.html](https://docs.deepwisdom.ai/v0.8/zh/guide/tutorials/multi%5C_agent%5C_101.html)

## 定义动作

我们可以定义三个具有各自动作的Role：

SimpleCoder 具有 SimpleWriteCode 动作，接收用户的指令并编写主要代码

SimpleTester 具有 SimpleWriteTest 动作，从 SimpleWriteCode 的输出中获取主代码并为其提供测试套件

SimpleReviewer 具有 SimpleWriteReview 动作，审查来自 SimpleWriteTest 输出的测试用例，并检查其覆盖范围和质量

In [1]:
import re
from metagpt.actions import Action, UserRequirement

# 构造写代码的动作

def parse_code(rsp):
    pattern = r"```python(.*)```"
    match = re.search(pattern, rsp, re.DOTALL)
    code_text = match.group(1) if match else rsp
    return code_text


class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction}.
    Return ```python your_code_here ```with NO other texts,
    your code:
    """
    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

2025-02-26 17:16:33.295 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /tmp/code/wow-agent/notebooks


In [2]:
# 构造写测试样例的动作
class SimpleWriteTest(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Write {k} unit tests using pytest for the given function, assuming you have imported it.
    Return ```python your_code_here ```with NO other texts,
    your code:
    """

    name: str = "SimpleWriteTest"

    async def run(self, context: str, k: int = 3):
        prompt = self.PROMPT_TEMPLATE.format(context=context, k=k)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

In [3]:
# 构造审查代码的动作
class SimpleWriteReview(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Review the test cases and provide one critical comments:
    """

    name: str = "SimpleWriteReview"

    async def run(self, context: str):
        prompt = self.PROMPT_TEMPLATE.format(context=context)

        rsp = await self._aask(prompt)

        return rsp

## 定义角色

在许多多智能体场景中，定义Role可能只需几行代码。对于SimpleCoder，我们做了两件事：

1. 使用 set\_actions 为Role配备适当的 Action，这与设置单智能体相同

2. 多智能体操作逻辑：我们使Role \_watch 来自用户或其他智能体的重要上游消息。回想我们的 SOP，SimpleCoder接收用户指令，这是由 MetaGPT 中的UserRequirement引起的Message。因此，我们添加了 self.\_watch([UserRequirement])。

In [4]:
# 构造写代码的角色
from metagpt.roles import Role
class SimpleCoder(Role):
    name: str = "Alice"
    profile: str = "SimpleCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._watch([UserRequirement])
        self.set_actions([SimpleWriteCode])

与上述相似，对于 SimpleTester，我们：

1. 使用 set\_actions 为SimpleTester配备 SimpleWriteTest 动作

2. 使Role \_watch 来自其他智能体的重要上游消息。回想我们的 SOP，SimpleTester从 SimpleCoder 中获取主代码，这是由 SimpleWriteCode 引起的 Message。因此，我们添加了 self.\_watch([SimpleWriteCode])。

3. 重写 \_act 函数，就像我们在智能体入门中的单智能体设置中所做的那样。在这里，我们希望SimpleTester将所有记忆用作编写测试用例的上下文，并希望有 5 个测试用例。


In [5]:
from metagpt.logs import logger
from metagpt.schema import Message

class SimpleTester(Role):
    name: str = "Bob"
    profile: str = "SimpleTester"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteTest])
        self._watch([SimpleWriteCode])
        # self._watch([SimpleWriteCode, SimpleWriteReview])  # feel free to try this too

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo

        # context = self.get_memories(k=1)[0].content # use the most recent memory as context
        context = self.get_memories()  # use all memories as context

        code_text = await todo.run(context, k=5)  # specify arguments
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg

In [6]:
# 按照相同的过程定义 SimpleReviewer：
class SimpleReviewer(Role):
    name: str = "Charlie"
    profile: str = "SimpleReviewer"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteReview])
        self._watch([SimpleWriteTest])

创建一个团队并添加角色

现在我们已经定义了三个 Role，是时候将它们放在一起了。我们初始化所有角色，设置一个 Team，并hire 它们。

运行 Team，我们应该会看到它们之间的协作！


In [7]:
import asyncio
from metagpt.team import Team

async def main(
    idea: str = "write a function that calculates the product of a list",
    investment: float = 3.0,
    n_round: int = 5,
):
    logger.info(idea)

    team = Team()
    team.hire(
        [
            SimpleCoder(),
            SimpleTester(),
            SimpleReviewer(),
        ]
    )

    team.invest(investment=investment)
    team.run_project(idea)
    await team.run(n_round=n_round)
await main()

2025-02-26 17:18:02.451 | INFO     | __main__:main:9 - write a function that calculates the product of a list
2025-02-26 17:18:02.714 | INFO     | metagpt.team:invest:90 - Investment: $3.0.
2025-02-26 17:18:02.717 | INFO     | metagpt.roles.role:_act:391 - Alice(SimpleCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def product_of_list(lst):
    product = 1
    for num in lst:
        product *= num
    return product
```

2025-02-26 17:18:03.940 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.
2025-02-26 17:18:03.946 | INFO     | __main__:_act:15 - Bob(SimpleTester): to do SimpleWriteTest(SimpleWriteTest)



```python
def test_product_of_list_empty():
    assert product_of_list([]) == 1

def test_product_of_list_single_element():
    assert product_of_list([5]) == 5

def test_product_of_list_positive_numbers():
    assert product_of_list([1, 2, 3, 4]) == 24

def test_product_of_list_negative_numbers():
    assert product_of_list([-1, -2, -3, -4]) == -24

def test_product_of_list_mixed_numbers():
    assert product_of_list([-1, 2, -3, 4]) == 24
```

2025-02-26 17:18:09.171 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.
2025-02-26 17:18:09.179 | INFO     | metagpt.roles.role:_act:391 - Charlie(SimpleReviewer): to do SimpleWriteReview(SimpleWriteReview)



Critical Comment:
While the test cases cover a variety of scenarios, including empty lists, single elements, positive numbers, negative numbers, and mixed numbers, there is a potential oversight in the test suite. The function `product_of_list` does not handle the case where the list contains a zero. The product of any list that includes zero should be zero, as multiplying by zero always results in zero. The current test suite does not include a test case to verify this behavior, which could lead to an undetected bug if the function is not implemented to handle zero correctly. Adding a test case for a list containing zero would be a critical addition to ensure the function

2025-02-26 17:18:13.093 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.


's robustness.
